# TPS Jan 2022 XGB + Cat + LGBM
__First notebook__
https://www.kaggle.com/kartushovdanil/tps-jan-22-eda-atboost-prophet
* In the first notebook, I noticed that it is better to train on the entire sample than on a k-fold.
* New idea: make out of year cross-validation, that will be better then the entire sample, and we can also find problem areas.
* Focus on the best solution to the find outliers and minimize residual
* Make residual analysis of 5 years
* Meta seed model? Done
_________________________________________________________
__soon__
* Not sure, but try minimize residual of april
* Covid?

In [ ]:
#================== TPS Jan 2022 =======================#
#----------------  import packages  -------------------#
import numpy as np 
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from cycler import cycler
import warnings
warnings.filterwarnings("ignore")
import math
import dateutil.easter as easter

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler

import lightgbm
import xgboost as xgb
from catboost import CatBoostRegressor
import pickle

from math import ceil, floor, sqrt

#------------------  load data   ---------------------#

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

train = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')

 
holiday_data = pd.read_csv('../input/public-and-unofficial-holidays-nor-fin-swe-201519/holidays.csv')

gdp_per_capita = pd.read_csv('../input/gdp-per-capita-finland-norway-sweden-201519/GDP_per_capita_2015_to_2019_Finland_Norway_Sweden.csv', index_col='year')

population = pd.read_csv('../input/population-20152019-finland-norway-sweden/population_2015-2019_Finland_Norway_Sweden.csv',index_col = 'year')

#!tree ../input/

In [ ]:
PSEUDO_STATUS = True
PSEUDO_DIR = "../input/pseudolabelsjan22/pseudo_labels_v0.csv"
if PSEUDO_STATUS == True:
    pseudo = pd.read_csv(PSEUDO_DIR)
    pseudo_test = test.copy()
    pseudo_test['num_sold'] = pseudo['num_sold']
    #pseudo_df = engineer(pseudo_test)

    
    
def preparate_df(df):
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['dayofmonth'] = df['date'].dt.days_in_month
    df['dayofyear'] = df['date'].dt.dayofyear
    df['weekday'] = df['date'].dt.weekday
    df['weekofyear'] = df['date'].dt.weekofyear
    #if 'num_sold' in df.columns:
        #df['log_num_sold'] = np.log(df['num_sold'])
    return df

train = preparate_df(train)
test = preparate_df(test)
pseudo_df = preparate_df(pseudo_test)
#pseudo_test = preparate_df(pseudo_test)



population.columns = gdp_per_capita.columns
population_dictionary = population.unstack().to_dict()
train["population"] = train.set_index(['country','year']).index.map(population_dictionary.get)
test["population"] = test.set_index(['country','year']).index.map(population_dictionary.get)
pseudo_df["population"] = pseudo_df.set_index(['country','year']).index.map(population_dictionary.get)



def holiday_features(holiday_df, df):
    
    fin_holiday = holiday_df.loc[holiday_df.country == 'Finland']
    swe_holiday = holiday_df.loc[holiday_df.country == 'Sweden']
    nor_holiday = holiday_df.loc[holiday_df.country == 'Norway']
    
    df['fin holiday'] = df.date.isin(fin_holiday.date).astype(int)
    df['swe holiday'] = df.date.isin(swe_holiday.date).astype(int)
    df['nor holiday'] = df.date.isin(nor_holiday.date).astype(int)
    
    df['holiday'] = np.zeros(df.shape[0]).astype(int)
    
    df.loc[df.country == 'Finland', 'holiday'] = df.loc[df.country == 'Finland', 'fin holiday']
    df.loc[df.country == 'Sweden', 'holiday'] = df.loc[df.country == 'Sweden', 'swe holiday']
    df.loc[df.country == 'Norway', 'holiday'] = df.loc[df.country == 'Norway', 'nor holiday']
    
    df.drop(['fin holiday', 'swe holiday', 'nor holiday'], axis=1, inplace=True)
    
    return df



holiday_features(holiday_data, train)
holiday_features(holiday_data, test)
holiday_features(holiday_data, pseudo_df)
holiday_features(holiday_data, pseudo_test)

gdp_dict = gdp_per_capita.unstack().to_dict()

# Create new 'gdp_per_capita' column
train['gdp_per_capita'] = train.set_index(['country', 'year']).index.map(gdp_dict.get)
test['gdp_per_capita']  = test.set_index(['country', 'year']).index.map(gdp_dict.get)
pseudo_df['gdp_per_capita']  = pseudo_df.set_index(['country', 'year']).index.map(gdp_dict.get)
pseudo_test['gdp_per_capita']  = pseudo_test.set_index(['country', 'year']).index.map(gdp_dict.get)

def fourier_features(df):
    # One-hot encoding (no need to encode the last categories)
    for country in ['Finland', 'Norway']:
        df[country] = df.country == country
        
    df['KaggleRama'] = df.store == 'KaggleRama'
    
    for product in ['Kaggle Mug', 'Kaggle Hat']:
        df[product] = df['product'] == product
    
    # Seasonal variations (Fourier series)
    # The three products have different seasonal patterns
    dayofyear = df.date.dt.dayofyear
    
    for k in range(1, 3):
        df[f'sin{k}'] = np.sin(dayofyear / 365 * 2 * math.pi * k)
        df[f'cos{k}'] = np.cos(dayofyear / 365 * 2 * math.pi * k)
        df[f'mug_sin{k}'] = df[f'sin{k}'] * df['Kaggle Mug']
        df[f'mug_cos{k}'] = df[f'cos{k}'] * df['Kaggle Mug']
        df[f'hat_sin{k}'] = df[f'sin{k}'] * df['Kaggle Hat']
        df[f'hat_cos{k}'] = df[f'cos{k}'] * df['Kaggle Hat']
        
    return df

fourier_features(train)
fourier_features(test)
fourier_features(pseudo_df)

le_dict = {feature: LabelEncoder().fit(train[feature]) for feature in ['country', 'product', 'store']}
def engineer_more(df):
    """Return a new dataframe with more engineered features"""
    new_df = df

    # End of year
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"dec{d}":
                                      (df.date.dt.month == 12) & (df.date.dt.day == d)
                                      for d in range(24, 32)}),
                        pd.DataFrame({f"n-dec{d}":
                                      (df.date.dt.month == 12) & (df.date.dt.day == d) & (df.country == 'Norway')
                                      for d in range(24, 32)}),
                        pd.DataFrame({f"f-jan{d}":
                                      (df.date.dt.month == 1) & (df.date.dt.day == d) & (df.country == 'Finland')
                                      for d in range(1, 14)}),
                        pd.DataFrame({f"jan{d}":
                                      (df.date.dt.month == 1) & (df.date.dt.day == d) & (df.country == 'Norway')
                                      for d in range(1, 10)}),
                        pd.DataFrame({f"s-jan{d}":
                                      (df.date.dt.month == 1) & (df.date.dt.day == d) & (df.country == 'Sweden')
                                      for d in range(1, 15)})],
                       axis=1)
    
    # May
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"may{d}":
                                      (df.date.dt.month == 5) & (df.date.dt.day == d) 
                                      for d in list(range(1, 10))}), #  + list(range(17, 25))
                        pd.DataFrame({f"may{d}":
                                      (df.date.dt.month == 5) & (df.date.dt.day == d) & (df.country == 'Norway')
                                      for d in list(range(19, 26))})],
                       axis=1)
    
    # June and July
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"june{d}":
                                      (df.date.dt.month == 6) & (df.date.dt.day == d) & (df.country == 'Sweden')
                                      for d in list(range(8, 14))}),
                        #pd.DataFrame({f"june{d}":
                        #              (df.date.dt.month == 6) & (df.date.dt.day == d) & (df.country == 'Norway')
                        #              for d in list(range(22, 31))}),
                        #pd.DataFrame({f"july{d}":
                        #              (df.date.dt.month == 7) & (df.date.dt.day == d) & (df.country == 'Norway')
                        #              for d in list(range(1, 3))})],
                       ],
                       axis=1)
    
    # Last Wednesday of June
    wed_june_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-06-24')),
                                         2016: pd.Timestamp(('2016-06-29')),
                                         2017: pd.Timestamp(('2017-06-28')),
                                         2018: pd.Timestamp(('2018-06-27')),
                                         2019: pd.Timestamp(('2019-06-26'))})
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"wed_june{d}": 
                                      (df.date - wed_june_date == np.timedelta64(d, "D")) & (df.country != 'Norway')
                                      for d in list(range(-4, 6))})],
                       axis=1)
    
    # First Sunday of November
    sun_nov_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-11-1')),
                                         2016: pd.Timestamp(('2016-11-6')),
                                         2017: pd.Timestamp(('2017-11-5')),
                                         2018: pd.Timestamp(('2018-11-4')),
                                         2019: pd.Timestamp(('2019-11-3'))})
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"sun_nov{d}": 
                                      (df.date - sun_nov_date == np.timedelta64(d, "D")) & (df.country != 'Norway')
                                      for d in list(range(0, 9))})],
                       axis=1)
    
    # First half of December (Independence Day of Finland, 6th of December)
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"dec{d}":
                                      (df.date.dt.month == 12) & (df.date.dt.day == d) & (df.country == 'Finland')
                                      for d in list(range(6, 14))})],
                       axis=1)

    # Easter
    easter_date = df.date.apply(lambda date: pd.Timestamp(easter.easter(date.year)))
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"easter{d}": 
                                      (df.date - easter_date == np.timedelta64(d, "D"))
                                      for d in list(range(-2, 11)) + list(range(40, 48)) + list(range(50, 59))})],
                       axis=1)
    for feature in ['country', 'product', 'store']:
        new_df[feature] = le_dict[feature].transform(df[feature])
    
    return new_df #.astype(np.float32)

train = engineer_more(train)
train['date'] = train.date
train['num_sold'] = train.num_sold.astype(np.float32)
test = engineer_more(test)
pseudo_df = engineer_more(pseudo_df)

def better_than_median(inputs, axis):
    """Compute the mean of the predictions if there are no outliers,
    or the median if there are outliers.

    Parameter: inputs = ndarray of shape (n_samples, n_folds)"""
    spread = inputs.max(axis=axis) - inputs.min(axis=axis) 
    spread_lim = 0.45
    print(f"Inliers:  {(spread < spread_lim).sum():7} -> compute mean")
    print(f"Outliers: {(spread >= spread_lim).sum():7} -> compute median")
    print(f"Total:    {len(inputs):7}")
    return np.where(spread < spread_lim,
                    np.mean(inputs, axis=axis),
                    np.median(inputs, axis=axis))

#preproc = StandardScaler()
#train = preproc.fit_transform(train)
#def smape(y_true, y_pred):
#    """SMAPE Loss"""
#    return np.abs(y_true - y_pred) / (y_true + np.abs(y_pred)) * 200
#

def smape(y_true, y_pred):
    """
    SMAPE Loss
    Parameters
    ----------
    y_true : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Ground truth (correct) target values.
    y_pred : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Estimated target values.
    Returns
    -------
    loss : float or ndarray of floats
        If multioutput is 'raw_values', then mean absolute error is returned
        for each output separately.
        If multioutput is 'uniform_average' or an ndarray of weights, then the
        weighted average of all output errors is returned.
        SMAPE output is non-negative floating point. The best value is 0.0.

    """
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

reduce_mem_usage(train)
reduce_mem_usage(test)
reduce_mem_usage(pseudo_df)

print(train.shape, test.shape, pseudo_df.shape)

print(train.columns)
print(test.columns)
print(pseudo_df.columns)

train['target'] = train['num_sold'] / train['gdp_per_capita']
pseudo_df['target'] = pseudo_df['num_sold'] / pseudo_df['gdp_per_capita']

FEATURES = [col for col in test.columns if col not in['row_id', 'date', 'num_sold', 'target']]
TARGET = ['target']
#
#    pseudo_df['date'] = pseudo_test.date 
#    pseudo_df['num_sold'] = pseudo_test.num_sold.astype(np.float32)
#    pseudo_df['target'] = np.log(pseudo_df['num_sold'] / pseudo_df['gdp'])
#
#    train_df = pd.concat([train_df,pseudo_df],axis=0)
#    train_df = train_df.reset_index(drop = True)

In [ ]:
missing_val = train.isnull().sum()
print(missing_val[missing_val > 0])
missing_val = test.isnull().sum()
print(missing_val[missing_val > 0])

In [ ]:
pseudo_df[FEATURES].head(5)

In [ ]:
lgb_params = {
        'objective': 'regression',
        'force_row_wise': True,
        'verbosity': -1,
        'seed': 1,
        'learning_rate': 0.03,
        'lambda_l1': 5e-05,
        'lambda_l2': 1e-06,
        'num_leaves': 20,
        'feature_fraction': 0.6,
        'bagging_fraction': 0.43,
        'bagging_freq': 5,
        'min_child_samples': 17,
        }                        

cat_params = {
        'eval_metric': 'SMAPE', 
        'use_best_model': True,
        'learning_rate': 0.04421730001498909,
        'depth': 6,
        'random_seed': 1,
        'l2_leaf_reg': 0.24960109471113703,
        'random_strength': 2.1314060037536735,
        'grow_policy': 'SymmetricTree',
        'max_bin': 406,
        'min_data_in_leaf': 77,
        'bootstrap_type': 'Bayesian',
        'bagging_temperature': 0.7392707417524894}

xgb_params = {
        'tree_method': 'hist',
        'grow_policy' : 'lossguide',
        'learning_rate': 0.03399878704233446,
        'max_depth': 5,
        'reg_alpha': 0.7814373604498039,
        'reg_lambda': 0.00018093104956619317,
        'max_delta_step': 2,
        'min_child_weight': 14,
        'colsample_bytree': 0.6489299778623602,
        'subsample': 0.6033298718112065,
        'max_leaves': 187,  
        }

xgb_params2 ={'lambda': 0.0012338191278124635, 'alpha': 3.284395992431614, 'eta': 0.09886834650237164, 'colsample_bytree': 0.9, 'subsample': 0.6, 'learning_rate': 0.018, 'n_estimators': 2000, 'max_depth': 5, 'min_child_weight': 3}
#def training_predictions():
#    all_score_list = []
#    lgb_score_list = []
#    cat_score_list = []
#    xgb_score_list = []
#    xgb_test = xgb.DMatrix(test[FEATURES])
#    
    

In [ ]:
train.head(5)

In [ ]:
def plot_importance(model):
    fea_imp = pd.DataFrame({'imp': model.feature_importances_, 'col': FEATURES})
    fea_imp = fea_imp.sort_values(['imp', 'col'], ascending=[True, False]).iloc[-30:]
    fea_imp.plot(kind='barh', x='col', y='imp', figsize=(10, 7), legend=None)
    plt.title(f'CatBoost - Feature Importance {year}')
    plt.ylabel('Features')
    plt.xlabel('Importance');
    
def plot_residual(xgb_pred, lgbm_pred, cat_pred, y_true, date):
    plt.rcParams['figure.dpi'] = 600
    colormap = ['#1DBA94','#1C5ED2', '#FFC300', '#C70039']
    plt.rc('axes', prop_cycle=(cycler('color', colormap)))
    background_color = '#f6f5f5'
    
    #all_pred = (cat_pred + xgb_pred + lgbm_pred).mean()


    fig, axs = plt.subplots(2, 1, figsize=(20, 10), facecolor='#f6f5f5')
    sns.kdeplot(y_true - xgb_pred,fill=True, alpha=0.7, label='xgb', ax=axs[0])
    sns.kdeplot(y_true - lgbm_pred,fill=True, alpha=0.7, label='lgbm', ax=axs[0])
    sns.kdeplot(y_true - cat_pred,fill=True, alpha=0.7, label='cat', ax=axs[0])
    #sns.kdeplot(y_true - all_pred,fill=True, alpha=0.7, label='all', ax=axs[0])
    axs[0].set_title('Distribution Residual', fontdict={'fontsize': 12, 'fontweight': 'bold'})
    axs[0].legend()

    sns.lineplot(date['date'], y_true - xgb_pred, alpha=0.7, label='xgb',ax=axs[1])
    sns.lineplot(date['date'], y_true - lgbm_pred, alpha=0.7, label='lgbm',ax=axs[1])
    sns.lineplot(date['date'], y_true - cat_pred, alpha=0.7, label='cat',ax=axs[1])
    #sns.lineplot(pseudo_df['date'], y_true - all_pred, alpha=0.7, label='all',ax=axs[1])
    axs[1].set_title('Plot Residual', fontdict={'fontsize': 14, 'fontweight': 'bold'})
    axs[1].legend()

    for i in range(2):
        for s in ["top","right"]:
            axs[i].spines[s].set_visible(False)
            axs[i].set_facecolor(background_color)
            axs[i].grid(which='major', axis='x', zorder=1, color='#EEEEEE', linewidth=0.4)
            axs[i].xaxis.offsetText.set_fontsize(4)
            axs[i].yaxis.offsetText.set_fontsize(4)
            axs[i].set_ylabel('')
            axs[i].set_xlabel('')
            axs[i].tick_params(labelsize=8, width=1)
            
    plt.plot()

In [ ]:
def fit_models(train, pseudo_df, test, log=False, year_split=False):
    cat_pred_df = pd.DataFrame()
    lgbm_pred_df = pd.DataFrame()
    cat_pred = []
    xgb_pred = []
    lgbm_pred = []      
    if year_split == True:
        year_date = [2015,2016,2017,2018]
    else:
        year_date = [1]
    for year in year_date:
        #----------------------- df ----------------------------
        if year_split==True:
            X_train = train[train['year'] != year][FEATURES]
            y_train = train[train['year'] != year][TARGET]
            X_valid = train[train['year'] == year][FEATURES]
            y_valid = train[train['year'] == year][TARGET]
            test = test[FEATURES]
            y_test = pseudo_df[TARGET]
        else:
            X_train = train[FEATURES]
            y_train = train[TARGET]
            X_valid = train[FEATURES]
            y_valid = train[TARGET]
            test = test[FEATURES]
            y_test = pseudo_df[TARGET]
        
        if log==True:
            y_train = np.log1p(y_train)
            y_valid = np.log1p(y_valid)
            y_test = np.log1p(y_test)
        
        
        lgb_data_train = lightgbm.Dataset(
                        X_train,
                        label = y_train)
        
        lgb_data_valid = lightgbm.Dataset(
                    X_valid, 
                    label = y_valid)
        
        lgb_data_test = lightgbm.Dataset(
                    test)

        #xgb_data_valid = xgb.DMatrix(
        #            X_valid,
        #            label = y_valid)
        #
        #xgb_data_train = xgb.DMatrix(
        #                X_train,
        #                label=y_train)
        #
        #xgb_data_test = xgb.DMatrix(
        #                test)
        
        #--------------------- models --------------------------
        cat_model = CatBoostRegressor(**cat_params) 
        cat_model.fit(X_train,y_train,eval_set =[(X_valid,y_valid)], verbose = 0, early_stopping_rounds = 200)
        
        #xgb_model = xgb.XGBRegressor(**xgb_params2)
        #xgb_model.fit(X_train, y_train,
        #                eval_set=[(X_train,y_train),(X_valid, y_valid)],
        #                verbose = False,
        #                early_stopping_rounds = 60)
        
        lgb_model = lightgbm.train(
                        lgb_params,
                        lgb_data_train,
                        num_boost_round=2000)
        
        #---------------------- plot ---------------------------
        #plot_importance(cat_model)
        #---------------------- predict ---------------------------
        cat_pred_train = cat_model.predict(X_train)
        cat_pred_valid = cat_model.predict(X_valid)
        cat_pred_test = cat_model.predict(test)
        cat_pred.append(cat_pred_test)
        
        #xgb_pred_train = xgb_model.predict(X_train)
        #xgb_pred_valid = xgb_model.predict(X_valid)
        #xgb_pred_test = xgb_model.predict(test)
        #xgb_pred.append(xgb_pred_test)
        
        lgbm_pred_train = lgb_model.predict(X_train)
        lgbm_pred_valid = lgb_model.predict(X_valid)
        lgbm_pred_test = lgb_model.predict(test)
        lgbm_pred.append(lgbm_pred_test)
        
        if log==True:
            cat_pred_train = np.expm1(cat_pred_train)
            cat_pred_valid = np.expm1(cat_pred_valid)
            cat_pred_test = np.expm1(cat_pred_test)
            #xgb_pred_train = np.expm1(xgb_pred_train)
            #xgb_pred_valid = np.expm1(xgb_pred_valid)
            #xgb_pred_test = np.expm1(xgb_pred_test)
            lgbm_pred_train = np.expm1(lgbm_pred_train)
            lgbm_pred_valid = np.expm1(lgbm_pred_valid)
            lgbm_pred_test = np.expm1(lgbm_pred_test)
            
            y_train = np.expm1(y_train)
            y_valid = np.expm1(y_valid)
            y_test = np.expm1(y_test)
        
        
        if year_split==True:
            cat_pred_df[f'y_{year}'] = cat_pred_test
            lgbm_pred_df[f'y_{year}'] = lgbm_pred_test
        else:
            cat_pred_df['y'] = cat_pred_test
            lgbm_pred_df['y'] = lgbm_pred_test
        
        #---------------------- score ---------------------------
        #train_score = smape(y_train, pred_train)
        cat_train_score = np.round(np.mean(smape(y_train.target, cat_pred_train)),4)
        #valid_score = smape(y_valid, pred_valid)
        cat_valid_score = np.round(np.mean(smape(y_valid.target, cat_pred_valid)),4)
        cat_test_score = np.round(np.mean(smape(y_test.target, cat_pred_test)),4)
        
        ##train_score = smape(y_train, pred_train)
        #xgb_train_score = np.round(np.mean(smape(y_train.target, xgb_pred_train)),4)
        ##valid_score = smape(y_valid, pred_valid)
        #xgb_valid_score = np.round(np.mean(smape(y_valid.target, xgb_pred_valid)),4)
        #xgb_test_score = np.round(np.mean(smape(y_test.target, xgb_pred_test)),4)
        
        #train_score = smape(y_train, pred_train)
        lgbm_train_score = np.round(np.mean(smape(y_train.target, lgbm_pred_train)),4)
        #valid_score = smape(y_valid, pred_valid)
        lgbm_valid_score = np.round(np.mean(smape(y_valid.target, lgbm_pred_valid)),4)
        lgbm_test_score = np.round(np.mean(smape(y_test.target, lgbm_pred_test)),4)
        
        #---------------------- output ---------------------------
        print('=====================================================')
        print(f'                    YEAR: != {year}')
        print(f'--------------------- CatBoost ---------------------')
        print(f' --------  train smape out of {year}: {cat_train_score}  --------  ')
        print(f' --------  valid smape {year}: {cat_valid_score}  --------  ')
        print(f' --------  test smape 2019: {cat_test_score}  --------  ')
        #print(f'-------------------- XGBBoost ----------------------')
        #print(f' --------  train smape out of {year}: {xgb_train_score}  --------  ')
        #print(f' --------  valid smape {year}: {xgb_valid_score}  --------  ')
        #print(f' --------  test smape 2019: {xgb_test_score}  --------  ')
        print(f'------------------ LGBM -----------------------')
        print(f' --------  train smape out of {year}: {lgbm_train_score}  --------  ')
        print(f' --------  valid smape {year}: {lgbm_valid_score}  --------  ')
        print(f' --------  test smape 2019: {lgbm_test_score}  --------  ')
        
        #plot_residual(xgb_pred_train, lgbm_pred_train, cat_pred_train, y_train.target, train[train['year'] != year])
        #print(xgb_pred_train.shape, lgbm_pred_train.shape, cat_pred_train.shape, y_train.shape, train[train['year'] != year])
        
    return cat_pred, xgb_pred, lgbm_pred, cat_pred_df, lgbm_pred_df
        
cat_pred, xgb_pred, lgbm_pred, cat_pred_df, lgbm_pred_df = fit_models(train, pseudo_df, test)


In [ ]:
ssb = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')

#ssb['num_sold'] = pd.DataFrame(cat_pred).mean()
#ssb.to_csv('submit_cat_mean_oof_nonlog_population_ysplitfalse.csv', index = False)
#ssb['num_sold'] = pd.DataFrame(xgb_pred).mean()
#ssb.to_csv('submit_xgb_mean_oof.csv', index = False)
#ssb['num_sold'] = pd.DataFrame(lgbm_pred).mean()
#ssb.to_csv('submit_lgbm_mean_oof.csv', index = False)
#ssb['num_sold'] = cat_pred[3] * 0.98
#ssb.to_csv('submit_cat_2015_2018_magic_num.csv', index = False)

In [ ]:
#cat_pred = 
all_pred = cat_pred + lgbm_pred 
#ssb['num_sold'] = pd.DataFrame(all_pred).mean()
#ssb.to_csv('submit_all_mean_oof.csv', index = False)

#ssb['num_sold'] = pd.DataFrame(all_pred).mean() * 0.99
#ssb.to_csv('submit_all_mean_oof_0dot99.csv', index = False)

In [ ]:
#all_pred = cat_pred[:2] + xgb_pred[:2] + lgbm_pred[:2]
#
#ssb['num_sold'] = pd.DataFrame(all_pred).mean()
#ssb.to_csv('submit_all_mean_oof_2m.csv', index = False)
#
#ssb['num_sold'] = pd.DataFrame(all_pred).mean() * 1.01
#ssb.to_csv('submit_all_mean_oof_1dot01.csv', index = False)

In [ ]:

plt.rcParams['figure.dpi'] = 600
colormap = ['#1DBA94','#1C5ED2', '#FFC300', '#C70039']
plt.rc('axes', prop_cycle=(cycler('color', colormap)))
background_color = '#f6f5f5'


fig, axs = plt.subplots(2, 1, figsize=(20, 10), facecolor='#f6f5f5')
#sns.kdeplot(pseudo_df.target - pd.DataFrame(xgb_pred).mean(),fill=True, alpha=0.7, label='xgb', ax=axs[0])
sns.kdeplot(pseudo_df.target - pd.DataFrame(lgbm_pred).mean(),fill=True, alpha=0.7, label='lgbm', ax=axs[0])
sns.kdeplot(pseudo_df.target - pd.DataFrame(cat_pred).mean(),fill=True, alpha=0.7, label='cat', ax=axs[0])
sns.kdeplot(pseudo_df.target - pd.DataFrame(all_pred).mean(),fill=True, alpha=0.7, label='all', ax=axs[0])
axs[0].set_title('Distribution Residual', fontdict={'fontsize': 14, 'fontweight': 'bold'})
axs[0].legend()

#sns.lineplot(pseudo_df['date'], pseudo_df.target - pd.DataFrame(xgb_pred).mean(), alpha=0.7, label='xgb',ax=axs[1])
sns.lineplot(pseudo_df['date'], pseudo_df.target - pd.DataFrame(lgbm_pred).mean(), alpha=0.7, label='lgbm',ax=axs[1])
sns.lineplot(pseudo_df['date'], pseudo_df.target - pd.DataFrame(cat_pred).mean(), alpha=0.7, label='cat',ax=axs[1])
sns.lineplot(pseudo_df['date'], pseudo_df.target - pd.DataFrame(all_pred).mean(), alpha=0.7, label='all',ax=axs[1])
axs[1].set_title('Plot Residual', fontdict={'fontsize': 14, 'fontweight': 'bold'})
axs[1].legend()

for i in range(2):
    for s in ["top","right"]:
        axs[i].spines[s].set_visible(False)
        axs[i].set_facecolor(background_color)
        axs[i].grid(which='major', axis='x', zorder=1, color='#EEEEEE', linewidth=0.4)
        axs[i].xaxis.offsetText.set_fontsize(4)
        axs[i].yaxis.offsetText.set_fontsize(4)
        axs[i].set_ylabel('')
        axs[i].set_xlabel('')
        axs[i].tick_params(labelsize=8, width=1)

In [ ]:
pseudo_df['residual'] = pseudo_df.target - pd.DataFrame(all_pred).mean()

In [ ]:
pseudo_df['residual'].sort_values(ascending=False)
print(pseudo_df['residual'].mean())
print(pseudo_df['residual'].std())

In [ ]:
#subm = test.gdp_per_capita * pd.DataFrame(all_pred).mean()
#ssb = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')
#ssb['num_sold'] = subm 
#ssb.to_csv('submit_cat_lgbm_gpd.csv', index = False)


In [ ]:
def mate_seed_model(num_seed):
    cat_meta_model_pred = pd.DataFrame()
    lgb_meta_model_pred = pd.DataFrame()
    for i in range(num_seed):
        lgb_params['seed'] = i
        cat_params['random_seed'] = i
        print('')
        print('')
        print('========================================================')
        print(f'===================== seed {i} =========================')
        print('========================================================')
        print('')
        print('')
        cat_pred, xgb_pred, lgbm_pred, cat_pred_df, lgbm_pred_df = fit_models(train, pseudo_df, test)
        cat_pred_df.columns = [col + f'_seed_{i}' for col in cat_pred_df.columns]
        lgbm_pred_df.columns = [col + f'_seed_{i}' for col in lgbm_pred_df.columns]
        cat_meta_model_pred = pd.concat([cat_meta_model_pred, cat_pred_df], axis=1)
        lgb_meta_model_pred = pd.concat([lgb_meta_model_pred, lgbm_pred_df], axis=1)
    return cat_meta_model_pred, lgb_meta_model_pred

In [ ]:
cat_meta_model_pred, lgb_meta_model_pred = mate_seed_model(10)

In [ ]:
#from functools import reduce
#from operator import add
cat_meta_model_pred = cat_meta_model_pred.mean(axis=1)
lgb_meta_model_pred = lgb_meta_model_pred.mean(axis=1)
all_meta_model_pred = (cat_meta_model_pred.mean(axis=1) + lgb_meta_model_pred.mean(axis=1))/2

In [ ]:

plt.rcParams['figure.dpi'] = 600
colormap = ['#1DBA94','#1C5ED2', '#FFC300', '#C70039']
plt.rc('axes', prop_cycle=(cycler('color', colormap)))
background_color = '#f6f5f5'


fig, axs = plt.subplots(2, 1, figsize=(20, 10), facecolor='#f6f5f5')
#sns.kdeplot(pseudo_df.target - pd.DataFrame(xgb_pred).mean(),fill=True, alpha=0.7, label='xgb', ax=axs[0])
sns.kdeplot(pseudo_df.target - lgb_meta_model_pred,fill=True, alpha=0.7, label='lgbm', ax=axs[0])
sns.kdeplot(pseudo_df.target - cat_meta_model_pred,fill=True, alpha=0.7, label='cat', ax=axs[0])
sns.kdeplot(pseudo_df.target - all_meta_model_pred,fill=True, alpha=0.7, label='all', ax=axs[0])
axs[0].set_title('Distribution Residual', fontdict={'fontsize': 14, 'fontweight': 'bold'})
axs[0].legend()

#sns.lineplot(pseudo_df['date'], pseudo_df.target - pd.DataFrame(xgb_pred).mean(), alpha=0.7, label='xgb',ax=axs[1])
sns.lineplot(pseudo_df['date'], pseudo_df.target - lgb_meta_model_pred, alpha=0.7, label='lgbm',ax=axs[1])
sns.lineplot(pseudo_df['date'], pseudo_df.target - cat_meta_model_pred, alpha=0.7, label='cat',ax=axs[1])
sns.lineplot(pseudo_df['date'], pseudo_df.target - all_meta_model_pred, alpha=0.7, label='all',ax=axs[1])
axs[1].set_title('Plot Residual', fontdict={'fontsize': 14, 'fontweight': 'bold'})
axs[1].legend()

for i in range(2):
    for s in ["top","right"]:
        axs[i].spines[s].set_visible(False)
        axs[i].set_facecolor(background_color)
        axs[i].grid(which='major', axis='x', zorder=1, color='#EEEEEE', linewidth=0.4)
        axs[i].xaxis.offsetText.set_fontsize(4)
        axs[i].yaxis.offsetText.set_fontsize(4)
        axs[i].set_ylabel('')
        axs[i].set_xlabel('')
        axs[i].tick_params(labelsize=8, width=1)

In [ ]:
subm = test.gdp_per_capita * all_meta_model_pred
ssb = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')
ssb['num_sold'] = subm 


In [ ]:

def geometric_round(arr):
    result_array = arr
    result_array = np.where(result_array < np.sqrt(np.floor(arr)*np.ceil(arr)), np.floor(arr), result_array)
    result_array = np.where(result_array >= np.sqrt(np.floor(arr)*np.ceil(arr)), np.ceil(arr), result_array)
    return result_array

ssb.num_sold = ssb.num_sold.apply(lambda x: geometric_round(x))

ssb.to_csv('submit_cat_lgbm_gpd_meta_geomround_nolog_nottsplit.csv', index = False)

In [ ]:
ssb

In [ ]:
#lgb_meta_model_pred

subm = test.gdp_per_capita * lgb_meta_model_pred
ssb = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')
ssb['num_sold'] = subm 


def geometric_round(arr):
    result_array = arr
    result_array = np.where(result_array < np.sqrt(np.floor(arr)*np.ceil(arr)), np.floor(arr), result_array)
    result_array = np.where(result_array >= np.sqrt(np.floor(arr)*np.ceil(arr)), np.ceil(arr), result_array)
    return result_array

ssb.num_sold = ssb.num_sold.apply(lambda x: geometric_round(x))

ssb.to_csv('submit_lgbm_gpd_meta_geomround_nolog_nottsplit.csv', index = False)

In [ ]:
#cat_meta_model_pred

subm = test.gdp_per_capita * cat_meta_model_pred
ssb = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')
ssb['num_sold'] = subm 


def geometric_round(arr):
    result_array = arr
    result_array = np.where(result_array < np.sqrt(np.floor(arr)*np.ceil(arr)), np.floor(arr), result_array)
    result_array = np.where(result_array >= np.sqrt(np.floor(arr)*np.ceil(arr)), np.ceil(arr), result_array)
    return result_array

ssb.num_sold = ssb.num_sold.apply(lambda x: geometric_round(x))

ssb.to_csv('submit_cat_gpd_meta_geomround_nolog_nottsplit.csv', index = False)